# Boston Airbnb Open Data Project


### Context:
Since 2008, guests and hosts have used Airbnb to travel in a more unique, personalized way. As part of the Airbnb Inside initiative, this dataset describes the listing activity of homestays in Boston, MA.

### Content:
The following Airbnb activity is included in this Boston dataset day

* Listings: including full descriptions and average review score
* Reviews: including unique id for each reviewer and detailed comments
* Calendar: including listing id and the price and availability for that day

If you are looking to really challenge yourself, data from Seattle and Boston Airbnb homes can be used to understand how much Airbnb homes are earning in certain time frames and areas. You can compare rates between the two cities, or try to understand if there is anything about the properties that helps you predict price. Can you find negative and positive reviews based on the text? This dataset requires a number of skills beyond those shown thus far in the course, but if you would like a challenge, this will certainly test your ability to work with messy, real-world data.

You can find additional Airbnb data at the link here(opens in a new tab).

Prepare data:
Gather necessary data to answer your questions
Handle categorical and missing data
Provide insight into the methods you chose and why you chose them
Analyze, Model, and Visualize
Provide a clear connection between your business questions and how the data answers them

I'm looking for answers for the below 3 questions:

1- month with highest earning in money

### 1.Data Cleaning & Exploration

In [2]:
# Let's start by importing our libraries

import matplotlib.pyplot as plt #used for plotting data 
import numpy as np #used for mathematical operations
import pandas as pd #used to loading CSV data
import seaborn as sns #used for plotting data 

In [3]:
# Suppress warning messages

import warnings
warnings.filterwarnings('ignore')

In [4]:
# Open the zip folder and check the files

import zipfile

z = zipfile.ZipFile('C:/Users/nsilman/DS_Citizen/Nano-Degree/archive.zip')

print (z.namelist())

['calendar.csv', 'listings.csv', 'reviews.csv']


In [5]:
# Read the datasets

df_calendar= pd.read_csv(z.open('calendar.csv'))
df_listings= pd.read_csv(z.open('listings.csv'))
df_reviews= pd.read_csv(z.open('reviews.csv'))

In [6]:
df_calendar.head(2)

,listing_id,date,available,price
0,12147973,2017-09-05,f,NaN
1,12147973,2017-09-04,f,NaN


In [7]:
df_listings.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,9.0,f,NaN,NaN,t,moderate,f,f,1,1.3


In [8]:
df_reviews.head(2)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,1178162,4724140,2013-05-21,4298113,Olivier,My stay at islam's place was really cool! Good...
1,1178162,4869189,2013-05-29,6452964,Charlotte,Great location for both airport and city - gre...


In [9]:
print("The shape of the calendar is {}".format(df_calendar.shape))
print("The shape of the listings is {}".format(df_listings.shape))
print("The shape of the reviews is {}".format(df_reviews.shape))

The shape of the calendar is (1308890, 4)
The shape of the listings is (3585, 95)
The shape of the reviews is (68275, 6)


Let's check the info for all datasets.

In [13]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308890 entries, 0 to 1308889
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   listing_id  1308890 non-null  int64 
 1   date        1308890 non-null  object
 2   available   1308890 non-null  object
 3   price       643037 non-null   object
dtypes: int64(1), object(3)
memory usage: 39.9+ MB


In [14]:
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3585 entries, 0 to 3584
Data columns (total 95 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                3585 non-null   int64  
 1   listing_url                       3585 non-null   object 
 2   scrape_id                         3585 non-null   int64  
 3   last_scraped                      3585 non-null   object 
 4   name                              3585 non-null   object 
 5   summary                           3442 non-null   object 
 6   space                             2528 non-null   object 
 7   description                       3585 non-null   object 
 8   experiences_offered               3585 non-null   object 
 9   neighborhood_overview             2170 non-null   object 
 10  notes                             1610 non-null   object 
 11  transit                           2295 non-null   object 
 12  access

In [15]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68275 entries, 0 to 68274
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   listing_id     68275 non-null  int64 
 1   id             68275 non-null  int64 
 2   date           68275 non-null  object
 3   reviewer_id    68275 non-null  int64 
 4   reviewer_name  68275 non-null  object
 5   comments       68222 non-null  object
dtypes: int64(3), object(3)
memory usage: 3.1+ MB


By taking a look at the .info method for the 3 datasets, it seems that listings dataset is the major one with all important information. We can merge the other 2 datasets to this one and then take a deeper look at the details. We have a common column in the 3 datasets which is "listing_id". I will use it to do the merge.

In [26]:
df_main= df_listings.join(df_calendar, how='left', on= 'id', rsuffix='_cal')
df_main.head(1)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,listing_id,date,available,price_cal
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,f,moderate,f,f,1,NaN,NaN,NaN,NaN,NaN


Next, I want to join the "reviews" dataset with them. But first, let's see if this data contains multiple rows for each listing to include each review separatey, or each listing has only 1 review. 

In [27]:
df_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,1178162,4724140,2013-05-21,4298113,Olivier,My stay at islam's place was really cool! Good...
1,1178162,4869189,2013-05-29,6452964,Charlotte,Great location for both airport and city - gre...
2,1178162,5003196,2013-06-06,6449554,Sebastian,We really enjoyed our stay at Islams house. Fr...
3,1178162,5150351,2013-06-15,2215611,Marine,The room was nice and clean and so were the co...
4,1178162,5171140,2013-06-16,6848427,Andrew,Great location. Just 5 mins walk from the Airp...


Apparently, each listing has multiple rows, every row contains a separate review with the the review details. Accordingly, we can't join this dataset with the other 2 at the moment. We need to analyse the data and aggregate it to have one row per listing, and then merge with the other main dataset.

Let's start with the df_reviews dataset

In [29]:
df_reviews.isna().sum()

listing_id        0
id                0
date              0
reviewer_id       0
reviewer_name     0
comments         53
dtype: int64

We have a small # of missing values in the comments column. I believe we can drop them because these comments are the reviews we are trying to analyze, we can't impute them.

In [30]:
df_reviews1=df_reviews.dropna(axis=0)
df_reviews1.isna().sum()

listing_id       0
id               0
date             0
reviewer_id      0
reviewer_name    0
comments         0
dtype: int64

Now let's analyse each review and classify it under 3 main categories [,,]

In [39]:
import inflect
import re
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from nltk.corpus import wordnet as wn

In [40]:
p = inflect.engine()
def convert_number(text):
    temp_str = text.split()
    new_string = []
 
    for word in temp_str:
        if word.isdigit():
            temp = p.number_to_words(word)
            new_string.append(temp)
        else:
            new_string.append(word)
 
    temp_str = ' '.join(new_string)
    return temp_str

In [41]:
nltk.download('stopwords')
english_stops = stopwords.words('english')
#add punctuation to stopwords
english_stops += list(string.punctuation)

[nltk_data] Error loading stopwords: <urlopen error Tunnel connection
[nltk_data]     failed: 407 Proxy Authentication Required>


In [42]:
print(nltk.data.path)

['C:\\Users\\nsilman/nltk_data', 'D:\\Data-GPT\\Anaconda\\nltk_data', 'D:\\Data-GPT\\Anaconda\\share\\nltk_data', 'D:\\Data-GPT\\Anaconda\\lib\\nltk_data', 'C:\\Users\\nsilman\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']


In [61]:
nltk.data.path = ['c:\nltk_data']

In [55]:
nltk.download('stopwords')


[nltk_data] Error loading stopwords: <urlopen error Tunnel connection
[nltk_data]     failed: 407 Proxy Authentication Required>


False

In [52]:
print(nltk.data.path)

['D:\\Data-GPT\\Anaconda\\nltk_data']


In [58]:
import nltk


In [59]:
from nltk.corpus import stopwords



In [60]:
nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error Tunnel connection
[nltk_data]     failed: 407 Proxy Authentication Required>


False

In [56]:
def text_processing(row):
    row['comments'] = row['comments'].lower()               ### convert text to lower case
    row['comments'] = re.sub(r"\s+[a-zA-Z]\s+", " ", row['comments'])   
    clean = re.compile('<.*?>')
    row['comments'] = re.sub(clean, '', row['comments'])
    row['comments'] = re.sub(r"\s+"," ", row['comments'], flags = re.I)
    row['comments'] = re.sub(r'^https?:\/\/.*[\r\n]*', '', row['comments'], flags=re.MULTILINE)
    row['comments'] = re.sub(r"\W", " ", row['comments'], flags=re.I)  ### remove all non-word characters
    row['comments'] = convert_number(row['comments'])       ### convert number to text
  
    review_words = word_tokenize(row['comments'])        ### get words from text
    review_words_no_stop = [item for item in review_words if item not in english_stops] ### removes stopwords & punctuation
    wordnet_lemmatizer = WordNetLemmatizer()
    words_lemmatization = [wordnet_lemmatizer.lemmatize(item) for item in review_words_no_stop] ### lemmatize words
    words_in_english = [item for item in words_lemmatization if len(wn.synsets(item)) != 0]
    
    unique_words = list(set(words_in_english))      ### removes duplicated words
    
    row['comments'] = " ".join(unique_words)             ### Concatenate all words
    return row['comments']

It seems that there are missing values in the "Price" column. Let's check its %

In [21]:
len(df_main)

3585

In [12]:
df_calendar.isna().sum()/len(df_calendar)

listing_id    0.000000
date          0.000000
available     0.000000
price         0.508716
dtype: float64

So there is a 50% missing values in the "Price" column. Before imputing these values, we need to merge this data with the other datasets to keep the listings that we are only interested in, which is the inner join between calendar & listings data.